In [1]:
import pygame

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from cellular_automaton.neighborhood import *
from cellular_automaton.factory import *
from cellular_automaton.rule import *
from cellular_automaton.display import *

import random
pygame.init()



(6, 0)

cellular_automaton library gives us a handy framework for creating cellular automatons. 
Every time you want to create one, you have to define a Rule, dimentions and neoighbourhood. Let us review this process on Starfall example.


First thing you have to do is defining a Rule, which is a class inheriting from class Rule, which defines following interface:


Rule gets a neighbourhood object in constructor. We will discuss neighbourhoods later.
```python
def __init__(self, neighborhood_: neighborhood.Neighborhood):
    self._neighborhood = neighborhood_
```


This is the method you have to implement. It describes the new state in every epoch.
By default it is set to have constant state. New state can depend on previous one and from the state of every neighbour

```python
@abc.abstractmethod
def evolve_cell(self, last_cell_state, neighbors_last_states):
    """ Calculates and sets new state of 'cell'.
    A cells evolution will only be called if it or at least one of its neighbors has changed last evolution_step.
    :param last_cell_state:         The cells state previous to the evolution step.
    :param neighbors_last_states:   The cells neighbors current states.
    :return: New state.             The state after this evolution step
    """
    return last_cell_state
```


Another method that has to be implemented is the initial state depending on cell location in the grid

```python
@abc.abstractmethod
def init_state(self, cell_coordinate):
    """ Will be called to initialize a cells state.
    :param cell_coordinate: Cells coordinate.
    :return: Iterable that represents the initial cell state
             Has to be compatible with 'multiprocessing.sharedctype.RawArray' when using multi processing.
    """
    return [0]
```


Library enables you to color the cell depending on the state. it can be usefull for example when visualizing forest on fire where "tree" can be alive, burning or already burned
```python
@abc.abstractmethod
def get_state_draw_color(self, current_state):
    """ Return the draw color for the current state """
    return [0, 0, 0]
```

There are also 4 predefined Neighbourhoods:

```python
class MooreNeighborhood(Neighborhood):
    """ Moore defined a neighborhood with a radius applied on a the non euclidean distance to other cells in the grid.
        Example:
            2 dimensions
            C = cell of interest
            N = neighbor of cell
            X = no neighbor of cell

                  Radius 1                     Radius 2
               X  X  X  X  X                N  N  N  N  N
               X  N  N  N  X                N  N  N  N  N
               X  N  C  N  X                N  N  C  N  N
               X  N  N  N  X                N  N  N  N  N
               X  X  X  X  X                N  N  N  N  N
    """

    def __init__(self, edge_rule: EdgeRule = EdgeRule.IGNORE_EDGE_CELLS, radius=1, dimension=2):
        super().__init__(tuple(_rel_neighbor_generator(dimension, radius, lambda rel_n: True)),
                         edge_rule)


class VonNeumannNeighborhood(Neighborhood):
    """ Von Neumann defined a neighborhood with a radius applied to Manhatten distance
        (steps between cells without diagonal movement).
        Example:
            2 dimensions
            C = cell of interest
            N = neighbor of cell
            X = no neighbor of cell

                  Radius 1                     Radius 2
               X  X  X  X  X                X  X  N  X  X
               X  X  N  X  X                X  N  N  N  X
               X  N  C  N  X                N  N  C  N  N
               X  X  N  X  X                X  N  N  N  X
               X  X  X  X  X                X  X  N  X  X
    """

    def __init__(self, edge_rule: EdgeRule = EdgeRule.IGNORE_EDGE_CELLS, radius=1, dimension=2):
        self.radius = radius
        super().__init__(tuple(_rel_neighbor_generator(dimension, radius, self.neighbor_rule)),
                         edge_rule)

    def neighbor_rule(self, rel_n):
        cross_sum = 0
        for ci in rel_n:
            cross_sum += abs(ci)
        return cross_sum <= self.radius


class RadialNeighborhood(Neighborhood):
    """ Neighborhood with a radius applied to euclidean distance + delta

        Example:
            2 dimensions
            C = cell of interest
            N = neighbor of cell
            X = no neighbor of cell

                  Radius 2                     Radius 3
            X  X  X  X  X  X  X          X  X  N  N  N  X  X
            X  X  N  N  N  X  X          X  N  N  N  N  N  X
            X  N  N  N  N  N  X          N  N  N  N  N  N  N
            X  N  N  C  N  N  X          N  N  N  C  N  N  N
            X  N  N  N  N  N  X          N  N  N  N  N  N  N
            X  X  N  N  N  X  X          X  N  N  N  N  N  X
            X  X  X  X  X  X  X          X  X  N  N  N  X  X
    """

    def __init__(self, edge_rule: EdgeRule = EdgeRule.IGNORE_EDGE_CELLS, radius=1, delta_=.25, dimension=2):
        self.radius = radius
        self.delta = delta_
        super().__init__(tuple(_rel_neighbor_generator(dimension, radius, self.neighbor_rule)),
                         edge_rule)

    def neighbor_rule(self, rel_n):
        cross_sum = 0
        for ci in rel_n:
            cross_sum += pow(ci, 2)

        return math.sqrt(cross_sum) <= self.radius + self.delta


class HexagonalNeighborhood(Neighborhood):
    """ Defines a Hexagonal neighborhood in a rectangular two dimensional grid:

        Example:
            Von Nexagonal neighborhood in 2 dimensions with radius 1 and 2
            C = cell of interest
            N = neighbor of cell
            X = no neighbor of cell

                  Radius 1                     Radius 2
               X   X   X   X   X           X   N   N   N   X
                 X   N   N   X               N   N   N   N
               X   N   C   N   X           N   N   C   N   N
                 X   N   N   X               N   N   N   N
               X   X   X   X   X           X   N   N   N   X


        Rectangular representation: Radius 1

          Row % 2 == 0            Row % 2 == 1
            N  N  X                 X  N  N
            N  C  N                 N  C  N
            N  N  X                 X  N  N

        Rectangular representation: Radius 2
          Row % 2 == 0            Row % 2 == 1
          X  N  N  N  X           X  N  N  N  X
          N  N  N  N  X           X  N  N  N  N
          N  N  C  N  N           N  N  C  N  N
          N  N  N  N  X           X  N  N  N  N
          X  N  N  N  X           X  N  N  N  X
    """

    def __init__(self, edge_rule: EdgeRule = EdgeRule.IGNORE_EDGE_CELLS, radius=1):
        neighbor_lists = [[(0, 0)],
                          [(0, 0)]]

        self.__calculate_hexagonal_neighborhood(neighbor_lists, radius)

        super().__init__(neighbor_lists, edge_rule)

    def __calculate_hexagonal_neighborhood(self, neighbor_lists, radius):
        for r in range(1, radius + 1):
            for i, n in enumerate(neighbor_lists):
                n = _grow_neighbours(n)
                n = self.__add_rectangular_neighbours(n, r, i % 2 == 1)
                n = sorted(n, key=(lambda ne: [ne[1], ne[0]]))
                n.remove((0, 0))
                neighbor_lists[i] = n

    def get_id_of_neighbor_from_relative_coordinate(self, rel_coordinate):
        raise NotImplementedError

    def _neighbors_generator(self, cell_coordinate):
        if not self._does_ignore_edge_cell_rule_apply(cell_coordinate):
            for rel_n in self._rel_neighbors[cell_coordinate[1] % 2]:
                yield from self._calculate_abs_neighbor_and_decide_validity(cell_coordinate, rel_n)

    @staticmethod
    def __add_rectangular_neighbours(neighbours, radius, is_odd):
        new_neighbours = []
        for x in range(0, radius + 1):
            if is_odd:
                x -= int(radius/2)
            else:
                x -= int((radius + 1) / 2)

            for y in range(-radius, radius + 1):
                new_neighbours.append((x, y))
        new_neighbours.extend(neighbours)
        return list(set(new_neighbours))
```


For given neighbourhoods you have to declare an edge rule.
There are three predefined rules:
```python
    IGNORE_EDGE_CELLS = 0
    IGNORE_MISSING_NEIGHBORS_OF_EDGE_CELLS = 1
    FIRST_AND_LAST_CELL_OF_DIMENSION_ARE_NEIGHBORS = 2
    ```

In [5]:
ALIVE = [1.0]
DEAD = [0]


class ConwaysRule(Rule):
    random_seed = random.seed(13)

    def init_state(self, cell_coordinate):
        rand = random.randrange(0, 16, 1)
        init = max(.0, float(rand - 14))
        return [init]

    def evolve_cell(self, last_cell_state, neighbors_last_states):
        new_cell_state = last_cell_state
        alive_neighbours = self.__count_alive_neighbours(neighbors_last_states)
        if last_cell_state == DEAD and alive_neighbours == 3:
            new_cell_state = ALIVE
        if last_cell_state == ALIVE and alive_neighbours < 2:
            new_cell_state = DEAD
        if last_cell_state == ALIVE and 1 < alive_neighbours < 4:
            new_cell_state = ALIVE
        if last_cell_state == ALIVE and alive_neighbours > 3:
            new_cell_state = DEAD
        return new_cell_state

    @staticmethod
    def __count_alive_neighbours(neighbours):
        an = []
        for n in neighbours:
            if n == ALIVE:
                an.append(1)
        return len(an)

    def get_state_draw_color(self, current_state):
        return [255 if current_state[0] else 0, 0, 0]


if __name__ == "__main__":
    neighborhood = MooreNeighborhood(EdgeRule.FIRST_AND_LAST_CELL_OF_DIMENSION_ARE_NEIGHBORS)
    ca = CAFactory.make_multi_process_cellular_automaton(dimension=[100, 100],
                                                         neighborhood=neighborhood,
                                                         rule=ConwaysRule,
                                                         processes=4)
    ca_window = CAWindow(cellular_automaton=ca, evolution_steps_per_draw=1)

    

In [7]:
"""
First, warmup exercise will be to edit the evolve_cell in starfall to take controlls over the "fall" direction. Make stars fly in oposite direction, then down and up.

"""


class StarfallRule(Rule):
    """ A basic cellular automaton that just copies one neighbour state so get some motion in the grid. """
    random_seed = random.seed(1000)

    def init_state(self, cell_coordinate):
        rand = random.randrange(0, 101, 1)
        init = max(.0, float(rand - 99))
        return [init]

    def evolve_cell(self, last_cell_state, neighbors_last_states):
        return self._get_neighbor_by_relative_coordinate(neighbors_last_states, (-1, -1))

    def get_state_draw_color(self, current_state):
        return [255 if current_state[0] else 0, 0, 0]


if __name__ == "__main__":
    neighborhood = MooreNeighborhood(EdgeRule.FIRST_AND_LAST_CELL_OF_DIMENSION_ARE_NEIGHBORS)
    
    ca = CAFactory.make_single_process_cellular_automaton(
        dimension=[100, 100],
        neighborhood=neighborhood,
        rule=StarfallRule)
    
    ca_window = CAWindow(cellular_automaton=ca, evolution_steps_per_draw=1)
    

Imagine you are a forester. You talked to a wise guy who figured out a formula of transitioning tree density, humidity and temperature into probability that a nearby tree will be set on fire by already burning one. Your job is to implement a cellular automata simulation of a burning forest.
Algorithm
States:
state 0 - alive tree
state 1 - burning tree
state 2 - dead tree

Transitions:
if state is alive and has any burning neighbours -it will be set on fire with probability equal to burning_factor: 0->1
otherwise it will still be alive 0->0

Burning tree in the next epoch will die, so always 1 ->2

Dead tree will remaid dead so always 2 ->2

Your job is to:
1. Implement evolve_cell method
2. Tweak the burning threshold to see what probability is suffient to burn down the whole forest and what probability doesn't harm the forest at all
3. let us investigate setting up the fire on purpose - change init state to:
   a) make fire start at a single, chosen point
   b) make fire start at single, chosen square area


In [ ]:

burning_factor = 0.35

class BurningForestRule(Rule):
    """ A basic cellular automaton that just copies one neighbour state so get some motion in the grid. """
    random_seed = random.seed(1000)

    def init_state(self, cell_coordinate):
        rand = random.randrange(0, 3001, 1)
        init = max(.0, float(rand - 2999))
        
        return [init] # possible init states are 0 - alive or 1 - burning

    def evolve_cell(self, last_cell_state, neighbors_last_states):
        # state is one-element list. One of [0] , [1] or 2
        # neighbors last states is a collection of states (1-element lists) of all neighbours 
        
        pass

    def get_state_draw_color(self, current_state):
        colors = {
            0: (0,100,0), #alive - green,
            1: (255,140,0), # burning- orange
            2: (43,29,14) # dead
        }
        return colors[current_state[0]]


if __name__ == "__main__":
    neighborhood = MooreNeighborhood(EdgeRule.FIRST_AND_LAST_CELL_OF_DIMENSION_ARE_NEIGHBORS)
    
    ca = CAFactory.make_single_process_cellular_automaton(
        dimension=[100, 100],
        neighborhood=neighborhood,
        rule=BurningForestRule)
    
    ca_window = CAWindow(cellular_automaton=ca, evolution_steps_per_draw=1)


## Brian's Brain

Brian's Brain consists of an infinite two-dimensional grid of cells, each cell may be in one of three states: on, dying, or off. Each cell is considered to have eight neighbors (the Moore neighborhood).

In each time step, a cell turns on if it was off but had exactly two neighbors that were on. All cells that were "on" go into the "dying" state, which is not counted as an "on" cell in the neighbor count, and prevents any cell from being born there. Cells that were in the dying state go into the off state.


In [ ]:
ON = [0]
DYING = [1]
OFF = [2]


class BrainsBrainRule(Rule):
    random_seed = random.seed(13)

    def init_state(self, cell_coordinate):
        rand = random.random()
        p = 0.35
        init = ON if rand >= p else OFF
        return init

    def evolve_cell(self, last_cell_state, neighbors_last_states):        
        new_cell_state = last_cell_state
        birth_threshold = 2
        alive_neighbours = self.__count_alive_neighbours(neighbors_last_states)
        # implement Brian's Brain cell evolution logic
        return new_cell_state

    @staticmethod
    def __count_alive_neighbours(neighbours):
        an = []
        for n in neighbours:
            if n == ON:
                an.append(1)
        return len(an)

    def get_state_draw_color(self, current_state):
        colors = {
            0: (255, 255, 255), 
            1: (18, 45, 153), 
            2: (0, 0, 0)
        }
        return colors[current_state[0]]

if __name__ == "__main__":    
    neighborhood = MooreNeighborhood(EdgeRule.FIRST_AND_LAST_CELL_OF_DIMENSION_ARE_NEIGHBORS)
    ca = CAFactory.make_multi_process_cellular_automaton(dimension=[100, 100],
                                                         neighborhood=neighborhood,
                                                         rule=BrainsBrainRule,
                                                         processes=4)
    ca_window = CAWindow(cellular_automaton=ca, evolution_steps_per_draw=1)

## Langton's ant

Squares on a plane are colored variously either black or white. We arbitrarily identify one square as the "ant". The ant can travel in any of the four cardinal directions at each step it takes. The "ant" moves according to the rules below:
* At a white square, turn 90° right, flip the color of the square, move forward one unit
* At a black square, turn 90° left, flip the color of the square, move forward one unit

In [ ]:
NORTH_WHITE = [0]
EAST_WHITE = [1]
SOUTH_WHITE = [2]
WEST_WHITE = [3]
NORTH_BLACK = [4]
EAST_BLACK = [5]
SOUTH_BLACK = [6]
WEST_BLACK = [7]
WHITE = [8]
BLACK = [9]


class LangtonsAntRule(Rule):

    def init_state(self, cell_coordinate):
        init = WEST_BLACK if cell_coordinate == (50, 50) else WHITE
        return init

    def evolve_cell(self, last_cell_state, neighbors_last_states):
        coordinates = [(0,-1),(1,0),(0,1),(-1,0)]
        matches_white = [EAST_WHITE, SOUTH_WHITE, WEST_WHITE, NORTH_WHITE]
        matches_black = [WEST_BLACK, NORTH_BLACK, EAST_BLACK, SOUTH_BLACK]
        for i in range(0, len(coordinates)):            
            neighbor = self._get_neighbor_by_relative_coordinate(neighbors_last_states, coordinates[i])
            if neighbor == matches_white[i] or neighbor == matches_black[i]:
                if last_cell_state == WHITE:
                    return matches_white[(i+1) % 4]
                elif last_cell_state == BLACK:
                    # where the ant should be facing if he turned left and came to this cell?
                    # look and the code two lines above for the hint
                    return matches_black[i]
        # handle the cases where ant moves from current cell to another cell
        return last_cell_state

    @staticmethod
    def __count_alive_neighbours(neighbours):
        an = []
        for n in neighbours:
            if n == ON:
                an.append(1)
        return len(an)

    def get_state_draw_color(self, current_state):
        if current_state[0] >= 0 and current_state[0] <= 3:
            return (255, 165, 156)
        elif current_state[0] >= 4 and current_state[0] <= 7:
            return (179, 16, 0)
        elif current_state[0] == 8:
            return (255, 255, 255)
        elif current_state[0] == 9:
            return (0, 0, 0)

if __name__ == "__main__":    
    neighborhood = VonNeumannNeighborhood(EdgeRule.FIRST_AND_LAST_CELL_OF_DIMENSION_ARE_NEIGHBORS)
    ca = CAFactory.make_multi_process_cellular_automaton(dimension=[100, 100],
                                                         neighborhood=neighborhood,
                                                         rule=LangtonsAntRule,
                                                         processes=4)
    ca_window = CAWindow(cellular_automaton=ca, evolution_steps_per_draw=1)

## Homework:

Add an additional ant (or a couple) to the system. Take a screenshot of the board with one ant and with two or more ants after a couple of iterations to compare how much the state of the system has changed after introducing additional ants.